In [5]:
import pandas as pd

import nltk
from nltk.corpus import wordnet 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

from pymorphy2 import MorphAnalyzer

import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\insps\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
file = 'C:/Users/insps/Downloads/Telegram Desktop/NLP_tokenized.csv'

In [8]:
data = pd.read_csv(file)

In [9]:
data.head(10)

,post_texts,all_comments,all_scores,tokenized_posts,tokenized_comments
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0,"['How', 'many', 'summer', 'Y', 'Combinator', '...","['Going', 'back', 'to', 'school', 'is', 'not',..."
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1,"['How', 'many', 'summer', 'Y', 'Combinator', '...","['There', 'will', 'invariably', 'be', 'those',..."
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2,"['How', 'many', 'summer', 'Y', 'Combinator', '...","['For', 'me', 'school', 'is', 'a', 'way', 'to'..."
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3,"['How', 'many', 'summer', 'Y', 'Combinator', '...","['I', 'guess', 'it', 'really', 'depends', 'on'..."
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4,"['How', 'many', 'summer', 'Y', 'Combinator', '...","['I', 'know', 'pollground', 'decided', 'to', '..."
5,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,0,"['CBS', 'acquires', 'last.fm', 'for', '$', '28...","['It', 'will', 'be', 'curious', 'to', 'see', '..."
6,CBS acquires last.fm for $280m,Does this mean that there's now a big-name com...,1,"['CBS', 'acquires', 'last.fm', 'for', '$', '28...","['Does', 'this', 'mean', 'that', ""there's"", 'n..."
7,CBS acquires last.fm for $280m,Also on BBC News: http://news.bbc.co.uk/1/low...,2,"['CBS', 'acquires', 'last.fm', 'for', '$', '28...","['Also', 'on', 'BBC', 'News', ':', 'http://new..."
8,CBS acquires last.fm for $280m,I don't understand what they do that is worth ...,3,"['CBS', 'acquires', 'last.fm', 'for', '$', '28...","['I', ""don't"", 'understand', 'what', 'they', '..."
9,CBS acquires last.fm for $280m,sold out too cheaply. given their leadership p...,4,"['CBS', 'acquires', 'last.fm', 'for', '$', '28...","['sold', 'out', 'too', 'cheaply', '.', 'given'..."


In [20]:
data[data.duplicated('all_comments')].sort_values('all_comments')

,post_texts,all_comments,all_scores,tokenized_posts,tokenized_comments
221360,"Gabe Newell Talks Linux Steam Client, Source E...",Even though Steam invites its own problems (it...,0,"['Gabe', 'Newell', 'Talks', 'Linux', 'Steam', ...","['Even', 'though', 'Steam', 'invites', 'its', ..."
199502,Dos & Don'ts Upon Taking a New Job,From the guidelines:If the original title begi...,2,"['Dos', '&', ""Don'ts"", 'Upon', 'Taking', 'a', ...","['From', 'the', 'guidelines', ':', 'If', 'the'..."
257622,"Lockitron Sells Over 9000 Units in First Week,...",Google cache: http://webcache.googleuserconten...,2,"['Lockitron', 'Sells', 'Over', '9000', 'Units'...","['Google', 'cache', ':', 'http://webcache.goog..."
286268,Pay Day Loan Rankings: The shady tale of Jooml...,Google cache: http://webcache.googleuserconten...,3,"['Pay', 'Day', 'Loan', 'Rankings', ':', 'The',...","['Google', 'cache', ':', 'http://webcache.goog..."
297144,How To Get Banned From PyCon,Google cache: http://webcache.googleuserconten...,4,"['How', 'To', 'Get', 'Banned', 'From', 'PyCon']","['Google', 'cache', ':', 'http://webcache.goog..."
298601,"Nasty Gal, an Online Start-Up, Is a Fast-Growi...",Paywall-less Google link to article: http://ww...,1,"['Nasty', 'Gal', ',', 'an', 'Online', 'Start-U...","['Paywall-less', 'Google', 'link', 'to', 'arti..."
355879,DMCA takedown on GPL-copyrighted code,http:&#x2F;&#x2F;webcache.googleusercontent.co...,4,"['DMCA', 'takedown', 'on', 'GPL-copyrighted', ...",['http://webcache.googleusercontent.com/search...
379798,Mistakes Programmers Make when Starting in Mac...,http:&#x2F;&#x2F;webcache.googleusercontent.co...,3,"['Mistakes', 'Programmers', 'Make', 'when', 'S...",['http://webcache.googleusercontent.com/search...
389267,Web Security Training Platform,http:&#x2F;&#x2F;webcache.googleusercontent.co...,2,"['Web', 'Security', 'Training', 'Platform']",['http://webcache.googleusercontent.com/search...
431916,Ask HN: What other sites do you read on a freq...,http:&#x2F;&#x2F;www.reddit.com&#x2F;r&#x2F;pr...,1,"['Ask', 'HN', ':', 'What', 'other', 'sites', '...",['http://www.reddit.com/r/programming']


In [10]:
data[data.all_comments.str.find('http') != -1].all_comments.iloc[2]

'I\'ve started to use Google Docs a lot. Trying Versionate, for text there is little advantage of either. GDocs seem a bit more peppy.I was reading a Paul Graham interview from a while ago:\nhttp://www.techcrunch.com/2006/09/02/an-interview-with-vc-pa...Here is an interesting quote:\n"\nI wouldn\'t advise competing with Google in things they\'re good at. So what is Google good at? As a first approximation, making things their own developers use at work. So they\'ll do a better job on an online calendar than a video sharing site, for example, because their employees are probably not supposed to be sitting watching videos at work.\n"He goes on to mention Google\'s size, which is probably another factor here.It would be interesting to hear more about what folks at Y-Combinator think about this direct assault on a growing Google service.'

In [11]:
stop_words = set(stopwords.words('english'))
morph = MorphAnalyzer()

In [12]:
corpus_train = data[['post_texts', 'all_comments']].copy()

In [13]:
def cleaner(text):
    text = re.sub(r'http\S+', '', text) 
    text = re.sub('[^a-zA-Z0-9]', ' ', text) 
    text = word_tokenize(text.lower()) 
    text = [morph.normal_forms(token)[0] for token in text 
            if token not in stop_words and len(token) > 2]
    text = " ".join(text)
    return text

In [14]:
corpus_train_clear = pd.DataFrame()
corpus_train_clear['post'] = corpus_train.post_texts.apply(cleaner)
corpus_train_clear['comments'] = corpus_train.all_comments.apply(cleaner)


KeyboardInterrupt: 

In [86]:
corpus_train_clear['score'] = data.all_scores

In [15]:
file_cl = 'C:/Users/insps/Downloads/Telegram Desktop/NLP_cleared.csv'
corpus_train_clear = pd.read_csv(file_cl)


In [16]:
corpus_train_clear.head(10)

,post,comments,score
0,many summer combinator fundees decided continu...,going back school identical giving founders ba...,0
1,many summer combinator fundees decided continu...,invariably see success set fall back original ...,1
2,many summer combinator fundees decided continu...,school way connected going real world entered ...,2
3,many summer combinator fundees decided continu...,guess really depends hungry much believe produ...,3
4,many summer combinator fundees decided continu...,know pollground decided back school getting co...,4
5,cbs acquires last 280m,curious see heads long run cbs tear fit image ...,0
6,cbs acquires last 280m,mean big name company fight repeal recent stre...,1
7,cbs acquires last 280m,also bbc news nice see london based hit headlines,2
8,cbs acquires last 280m,understand worth 70m year,3
9,cbs acquires last 280m,sold cheaply given leadership position ask lea...,4


In [90]:
corpus_train_clear.to_csv('NLP_cleared.csv', index= False)

In [17]:
corpus_train_clear.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440535 entries, 0 to 440534
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   post      440135 non-null  object
 1   comments  439139 non-null  object
 2   score     440535 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 10.1+ MB


In [19]:
comments_and_posts_cleared = corpus_train_clear.copy()

In [ ]:
count_vect = CountVectorizer()
matrix_post = count_vect.fit_transform(comments_and_posts_cleared.post.astype('U'))
matrix_comments = count_vect.transform(comments_and_posts_cleared.comments.astype('U'))

In [22]:
len(count_vect.vocabulary_)

220072

In [24]:
matrix_comments.shape

(440535, 220072)

In [107]:
tfidf_vect = TfidfVectorizer(min_df=10)

In [3]:
matrix_post_ti = count_vect.fit_transform(comments_and_posts_cleared.post)
matrix_comments_ti = count_vect.transform(comments_and_posts_cleared.comments)

NameError: name 'count_vect' is not defined

In [115]:
matrix_comments_ti.shape

(440535, 220072)

In [26]:
matrix_comments[0]

<1x220072 sparse matrix of type '<class 'numpy.int64'>'
	with 62 stored elements in Compressed Sparse Row format>

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(5):
    print(cosine_similarity(matrix_post[0], matrix_comments[i]))

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 35357 while Y.shape[1] == 220072